## Import Packages

In [ ]:
# import data processing and visualisation libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import image processing libraries
import cv2
import skimage
from skimage.transform import resize

# import tensorflow and keras
import tensorflow as tf
from tensorflow import keras
import os

## Load Data

In [ ]:
#기본 Parameter 설정
BATCHSIZE = 25
imageSize = 60
target_dims = (imageSize, imageSize, 3)
num_classes = 11 #0~10까지 총 11개
train_len = 1534 #총 파일 수

train_dir = "../input/full/" #train 폴더 경로 지정(끝에 / 까지 쳐야함)

In [ ]:
from tqdm import tqdm #진행상황바 표시해주는 패키지

def get_data(folder, data_len): #data불러오는 함수
    
    X = np.empty((data_len, imageSize, imageSize, 3), dtype=np.float32)
    y = np.empty((data_len,), dtype=np.int)
    cnt = 0

    for folderName in os.listdir(folder): #기존 ASL alphabet용 코드를 약간 변형시켜줌
        if not folderName.startswith('.'):
            if folderName in ['00']: #label이 0부터 시작하게 하려고 0 사진 다운 받아둠
                label = 0
            elif folderName in ['01']:
                label = 1
            elif folderName in ['02']:
                label = 2
            elif folderName in ['03']:
                label = 3
            elif folderName in ['04']:
                label = 4
            elif folderName in ['05']:
                label = 5
            elif folderName in ['06']:
                label = 6
            elif folderName in ['07']:
                label = 7
            elif folderName in ['08']:
                label = 8
            elif folderName in ['09']:
                label = 9
            elif folderName in ['10']:
                label = 10
            for image_filename in tqdm(os.listdir(folder + folderName)):
              img_file = cv2.imread(folder + folderName + '/' + image_filename)
              if img_file is not None:
                img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3)) #image 사이즈 조정
                img_arr = np.asarray(img_file)
                
                X[cnt] = img_arr
                y[cnt] = label
                cnt += 1

    return X,y

In [ ]:
X_train, y_train = get_data(train_dir,train_len)

In [ ]:
#pickle 파일 load

import pickle

with open("../pickle/X_train_220602_imagesize_60_full","rb") as file :
    X_train=pickle.load(file)
    
with open("../pickle/y_train_220602_imagesize_60_full","rb") as file2 :
    y_train=pickle.load(file2)

In [ ]:
#data 불러온 결과 shape출력(데이터수, imageSize, imageSize, 3) : get_data함수에서 설정해준 항목
print("The shape of X_train is : ", X_train.shape)
print("The shape of y_train is : ", y_train.shape)
print("The shape of one image is : ", X_train[0].shape)

In [ ]:
#혹시 몰라 나중에 쓸 수도 있어 pickle파일 만들어봄(건너뛰어도 무관)
import pickle

X_train_220602_imagesize_60_full=X_train
y_train_220602_imagesize_60_full=y_train


with open("../pickle/X_train_220602_imagesize_60_full","wb") as file :
    pickle.dump(X_train_220602_imagesize_60_full,file)
with open("../pickle/y_train_220602_imagesize_60_full","wb") as file :
    pickle.dump(y_train_220602_imagesize_60_full,file)
    

In [ ]:
#예시로 이미지 출력
plt.imshow(X_train[0])
plt.show()

In [ ]:
#Making copies of original data
X_data = X_train
y_data = y_train

In [ ]:
#있는 데이터로 train, test split해주는 함수, test_size에서 비율 조정 가능
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3,random_state=42,stratify=y_data)

In [ ]:
#train,test split된 것 출력해서 확인
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Modeling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [ ]:
ResNet_transfer_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=target_dims)

In [ ]:
ResNet_transfer_model.trainable=False

In [ ]:
#Early Stopping함수를 이용해서 patience수가 넘어가면 자동으로 Early Stopping되게 함
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
ResNet_model_tr = Sequential([
    ResNet_transfer_model, 
    Flatten(), 
    Dense(64, activation='relu'), 
    Dense(11, activation='softmax')
])

In [ ]:
#Compiling
ResNet_model_tr.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Model fitting
ResNet_model_tr.fit(X_train, y_train,
          epochs=100,
          batch_size=BATCHSIZE,
          verbose=2,
          validation_data=(X_test, y_test),
          callbacks=[early_stop])

## Result

In [ ]:
#Model metrics
metrics = pd.DataFrame(VGG16_model_tr.history.history)
print("The model metrics are")
metrics

In [ ]:
#저장된 accuracy 그래프로 출력
metrics[['accuracy','val_accuracy']].plot()
plt.show()

In [ ]:
#저장된 loss 그래프로 출력
metrics[['loss','val_loss']].plot()
plt.show()

In [ ]:
#Model evaluation
ResNet_model_tr.evaluate(X_test,y_test,verbose=0)

In [ ]:
y_predictions=ResNet_model_tr.predict(X_test)
predictions=y_predictions.argmax(axis=1)

In [ ]:
#Report
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,predictions))

In [ ]:
#Confusion matrix로 표시
plt.figure(figsize=(12,12))
sns.heatmap(confusion_matrix(y_test,predictions))
plt.show()